<a href="https://colab.research.google.com/github/HSE-LAMBDA/mldm-2019/blob/master/day-2/contest/NoFreeLunch_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from multiprocessing import Process
import time
import pickle
import importlib
from copy import deepcopy

In [0]:
def check_sample(c1, c2, size):
  assert isinstance(c1, np.ndarray), "c1 is not an ndarray"
  assert isinstance(c2, np.ndarray), "c2 is not an ndarray"
  assert c1.ndim == c2.ndim == 2, "got non-2d array"
  assert c1.shape[0] == c2.shape[0] == size, "generate returned an array of wrong length"
  assert c1.shape[1] == c2.shape[1] == 2, "generate returned an array with wrong number of features (expected 2), got: {} and {}".format(c1.shape[1], c2.shape[1])

def merge_ds(c0, c1):
  features = np.concatenate([c0, c1], axis=0)
  answers  = np.ones(shape=(len(c0) + len(c1), 1), dtype=float)
  answers[:len(c0)] = 0.
  result = np.concatenate([features, answers], axis=1)
  np.random.shuffle(result)
  return result

def run_test(dataset, model):
  dataset = dataset.copy()
  data_train, data_test = train_test_split(dataset, test_size=0.2)
  model.fit(data_train[:,:-1], data_train[:,-1])
  predictions = np.array(model.predict(data_test[:,:-1])).squeeze().astype(float)
  assert isinstance(predictions, np.ndarray), "your model predictions must be an ndarray"
  assert predictions.ndim == 1, "your model predictions must be a 1d vector"
  assert len(predictions) == len(data_test), "the length of predictions doesn't equal the lenght of input"

  accuracy = (predictions == data_test[:,-1]).mean()
  return accuracy

In [0]:
def run_test_wrapper(dataset, model, filename):
  try:
    result = run_test(dataset, model)
  except Exception as e:
    result = str(e)
  with open(filename, 'wb') as f:
    pickle.dump(result, f)

def run_test_limited(dataset, model, limit=10):
  filename = 'tmp.pickle'
  p = Process(target=run_test_wrapper, args=(dataset, model, filename))
  p.start()
  p.join(timeout=limit)
  p.terminate()
  time.sleep(1)
  try:
    with open(filename, 'rb') as f:
      result = pickle.load(f)
    os.remove(filename)
    return result
  except:
    pass

  return "Timeout"

In [0]:
TEST_LIMIT = 10
SAMPLE_HALF_SIZE = 400

class User:
  counter = 0
  def __init__(self, name, gen_code, model_code):
    User.counter += 1
    self.name = name
    self.gen_code = gen_code
    self.model_code = model_code
    self.valid = True
    self.scores = {}
    self.scores_T = {}
    self.error = None
    
    fname_gen = "tmp_script_gen_{}".format(User.counter)
    fname_gen_py = fname_gen + '.py'
    if os.path.isfile(fname_gen_py):
      os.remove(fname_gen_py)

    with open(fname_gen_py, 'w') as f:
      f.write(gen_code)

    fname_model = 'tmp_script_model_{}'.format(User.counter)
    fname_model_py = fname_model + '.py'
    if os.path.isfile(fname_model_py):
      os.remove(fname_model_py)
    with open(fname_model_py, 'w') as f:
      f.write(model_code)
      
    try:
      self.m_gen   = importlib.import_module(fname_gen)
      self.m_model = importlib.import_module(fname_model)
      self.generate = self.m_gen.generate
      self.model    = self.m_model.model
      if isinstance(self.model, type):
        self.model = self.model()
      
      c0 = self.generate(SAMPLE_HALF_SIZE, 0).astype(float)
      c1 = self.generate(SAMPLE_HALF_SIZE, 1).astype(float)
      check_sample(c0, c1, SAMPLE_HALF_SIZE)
      self.dataset = merge_ds(c0, c1)

    except Exception as e:
      self.error = str(e)
      self.generate = None
      self.model = None
      self.dataset = None
      self.valid = False

    if self.valid:
      for i in range(4):
        test_result = run_test_limited(self.dataset, self.get_model(), TEST_LIMIT)
        if isinstance(test_result, str):
          self.error = test_result
          self.valid = False
        elif test_result < 0.55:
          self.error = 'the model failed attempt #{} (score = {}) on its own dataset'.format(i + 1, test_result)
          self.valid = False
        else:
          self.error = None
          self.valid = True
          self.scores[self.name] = test_result
          self.scores_T[self.name] = test_result
          break

  def get_model(self):
    return deepcopy(self.model)

In [0]:
import requests
import pandas as pd
from io import StringIO

key = '1Vvv4I4n8p8N3HKk_gZqajD07MuU8hBRcnz27B22D_Xc'
format_str = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'

data = requests.get(format_str.format(key)).content
df = pd.read_csv(StringIO(data.decode()))

In [6]:
df

,Отметка времени,Your name,Data generating function,Model
0,17.09.2019 15:19:53,Baseline_test1,"import numpy as np\n\ndef generate(n, c):\n ...",from sklearn.neighbors import KNeighborsClassi...
1,17.09.2019 15:23:15,Baseline_test2,"def generate(n, c):\n if c == 0:\n ...",from sklearn.neighbors import KNeighborsClassi...
2,17.09.2019 15:24:53,Baseline_test3,"import numpy as np\n\ndef generate(n, c):\n ...",from sklearn.neighbors import KNeighborsClassi...
3,17.09.2019 17:32:16,Boris,"def generate(n, c):\n if c == 0:\n r...",from lightgbm import LGBMClassifier\nfrom skle...
4,17.09.2019 17:34:01,Boris (fixed),"import numpy as np\n\ndef generate(n, c):\n ...",from lightgbm import LGBMClassifier\nfrom skle...
5,17.09.2019 17:46:24,Ravil Khairullin,import random\nfrom operator import xor\ndef g...,model = LogisticRegression()
6,17.09.2019 17:47:32,Khairullin Ravil (2),import random\nfrom operator import xor\ndef g...,from sklearn.linear_model import LogisticRegre...
7,18.09.2019 9:55:37,Eric George Parakal,"def generate(n, c):\n \n if c == 0:\n ...",import xgboost as xgb\n\nxgb_model = xgb.XGBCl...
8,19.09.2019 15:05:23,Ruben Freire,"import numpy as np\ndef generate(n, c):\n ...",from sklearn.neighbors import KNeighborsClassi...
9,19.09.2019 22:51:38,yulia_gurova_s1,"import numpy as np\ndef generate(n,c):\n if c...",from sklearn.neighbors import KNeighborsClassi...


In [0]:
from tqdm import tqdm

In [0]:
def experiment():
  users = []

  for i, entry in df[['Your name', 'Data generating function', 'Model']].iterrows():
    if entry.isna().any(): continue
    name, code_gen, code_model = entry
    if any([name == u.name for u in users]):
      continue

    start_time = time.time()
    users.append(User(name, code_gen, code_model))
    elapsed = time.time() - start_time
    print("User", name, "tested in", elapsed, "seconds")


  for u1 in tqdm(users):
    if not u1.valid:
      continue

    for u2 in users:
      if not u2.valid:
        continue
      
      if u2.name in u1.scores:
        continue
      
      test_result = run_test_limited(u2.dataset, u1.get_model(), TEST_LIMIT)
      if test_result == 'Timeout':
        u1.scores  [u2.name] = -1.
        u2.scores_T[u1.name] = -1.
      else:
        u1.scores  [u2.name] = test_result
        u2.scores_T[u1.name] = test_result
    

  for u in users:
    print(u.name, u.valid, u.error)
    print("  scores:")
    for k, v in u.scores.items():
      try:
        str_v = '{:.3}'.format(v)
      except:
        str_v = str(v)
      print('    {:20} : {}'.format(k, str_v))

  scores = pd.DataFrame([(u.name,
                          np.mean(list(u.scores  .values())),
                          np.mean(list(u.scores_T.values()))) for u in users if u.valid],
                         columns=['username', 'mean_model_score', 'mean_dataset_loss'])
  return scores

In [0]:
import pickle

In [10]:
results = []
for i in range(10):
  print('Running experiment #', i)
  results.append(experiment())

with open("results.pkl", 'wb') as f:
  pickle.dump(results, f)

Running experiment # 0
User Baseline_test1 tested in 1.1803760528564453 seconds
User Baseline_test2 tested in 0.001486063003540039 seconds
User Baseline_test3 tested in 1.0316426753997803 seconds
User Boris tested in 0.05986833572387695 seconds
User Boris (fixed) tested in 1.1137495040893555 seconds
User Ravil Khairullin tested in 0.0020682811737060547 seconds
User Khairullin Ravil (2) tested in 0.0042040348052978516 seconds
User Eric George Parakal tested in 0.13057374954223633 seconds
User Ruben Freire tested in 0.0014677047729492188 seconds
User yulia_gurova_s1 tested in 0.0021152496337890625 seconds
User Ruben Freire (2) tested in 1.0354244709014893 seconds
User yulia_gurova_s2 tested in 1.025989055633545 seconds


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


User Anna Beketova tested in 4.120912075042725 seconds
User Anna Graessel tested in 1.0375969409942627 seconds
User Fouzi Takelait tested in 0.002881288528442383 seconds
User Eric_George_Parakal_2 tested in 1.2408881187438965 seconds


  0%|          | 0/17 [00:00<?, ?it/s]

User Fouzi_v3 tested in 4.117706537246704 seconds


...

User Anna Beketova tested in 4.11436128616333 seconds
User Anna Graessel tested in 1.036839246749878 seconds
User Fouzi Takelait tested in 0.0021746158599853516 seconds
User Eric_George_Parakal_2 tested in 1.0862007141113281 seconds


  0%|          | 0/17 [00:00<?, ?it/s]

User Fouzi_v3 tested in 4.124982595443726 seconds


100%|██████████| 17/17 [00:43<00:00,  2.57s/it]

Baseline_test1 True None
  scores:
    Baseline_test1       : 0.613
    Baseline_test3       : 0.806
    Boris (fixed)        : 0.694
    Ruben Freire (2)     : 1.0
    yulia_gurova_s2      : 0.531
    Anna Graessel        : 0.938
    Eric_George_Parakal_2 : 0.731
Baseline_test2 False name 'np' is not defined
  scores:
Baseline_test3 True None
  scores:
    Baseline_test3       : 0.8
    Baseline_test1       : 0.55
    Boris (fixed)        : 0.694
    Ruben Freire (2)     : 1.0
    yulia_gurova_s2      : 0.531
    Anna Graessel        : 0.938
    Eric_George_Parakal_2 : 0.731
Boris False name 'np' is not defined
  scores:
Boris (fixed) True None
  scores:
    Boris (fixed)        : 0.675
    Baseline_test1       : 0.631
    Baseline_test3       : 0.812
    Ruben Freire (2)     : 0.994
    yulia_gurova_s2      : 0.525
    Anna Graessel        : 0.931
    Eric_George_Parakal_2 : 0.675
Ravil Khairullin False name 'LogisticRegression' is not defined
  scores:
Khairullin Ravil (2) False nam

In [41]:
with open('results.pkl', 'rb') as f:
  results = pickle.load(f)

for i, r in enumerate(results):
  r.columns = ["{}_{}".format(c, i) if c != 'username' else c for c in r.columns]

result = pd.concat([r.set_index('username') for r in results],
                   axis=1, sort=True, copy=True)

model_cols   = [c for c in result.columns if 'mean_model_score'  in c]
dataset_cols = [c for c in result.columns if 'mean_dataset_loss' in c]

result[model_cols  ].fillna(0.5, inplace=True)
result[dataset_cols].fillna(1.0, inplace=True)

result['total_model_score'  ] = result[model_cols  ].mean(axis=1)
result['total_dataset_score'] = result[dataset_cols].mean(axis=1)

final_scores = result[['total_model_score', 'total_dataset_score']]

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [43]:
final_scores.sort_values('total_model_score', ascending=False)

,total_model_score,total_dataset_score
yulia_gurova_s2,0.794993,0.569157
Anna Graessel,0.741528,0.898845
Baseline_test1,0.739879,0.572168
Boris (fixed),0.739070,0.670723
Baseline_test3,0.733258,0.793689
Anna Beketova,0.732639,0.572005
Fouzi_v3,0.730649,0.546802
Ruben Freire (2),0.729907,0.991687
Eric_George_Parakal_2,0.718700,0.838952


In [45]:
final_scores.sort_values('total_dataset_score', ascending=True)

,total_model_score,total_dataset_score
Fouzi_v3,0.730649,0.546802
yulia_gurova_s2,0.794993,0.569157
Anna Beketova,0.732639,0.572005
Baseline_test1,0.739879,0.572168
Boris (fixed),0.739070,0.670723
Baseline_test3,0.733258,0.793689
Eric_George_Parakal_2,0.718700,0.838952
Anna Graessel,0.741528,0.898845
Ruben Freire (2),0.729907,0.991687
